> **How to run this notebook (command-line)?**
1. Install the `reinvent_shared.v2.1` environment:
`conda env create -f reinvent_shared.yml`
2. Activate the environment:
`conda activate reinvent_shared.v2.1`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 2.0`: reinforcement learning selectivity demo
This illustrates the use of a selectivity score component. This component is normally used in a scenario where ligands are designed to bind to a concrete target and at the same time should not be binding to one or more off-targets. The example here looks at a single off-target case.

We will demonstrate how to set up a `REINVENT` run that optimizes molecules to be selective against _Aurora_ kinase by minimizing at the same time their affinity to _B RAF_ kinase.

NOTE: There is a detailed reasoning for each code block provided in the `Reinforcement Learning Demo` notebook.


## 1. Set up the paths
_Please update the following code block such that it reflects your system's installation and execute it._

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Projects/Publications/2020/2020-04_REINVENT_2.0/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent_shared.v2.1")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

In [2]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("/home/kjrw606/PycharmProjects/github_repositories/reinvent")
reinvent_env = os.path.expanduser("/home/kjrw606/miniconda3/envs/reinvent_shared.v2.1")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## 2. Setting up the configuration 
In the cells below we will build a nested dictionary object that will be eventually converted to JSON file which in turn will be consumed by `REINVENT`. 
You can find this file in your `output_dir` location.

### A) Declare the run type

In [3]:
# initialize the dictionary
configuration = {
    "version": 2,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning"   # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model"
}

### B) Sort out the logging details
This includes `resultdir` path where the results will be produced.

Also: `REINVENT` can send custom log messages to a remote location. We have retained this capability in the code. if the `recipient` value differs from `"local"` `REINVENT` will attempt to POST the data to the specified `recipient`. 

In [4]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://0.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_frequency": 10,               # log every x-th steps
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "resultdir": os.path.join(output_dir, "results"),         # will hold the compounds (SMILES) and summaries
    "job_name": "Reinforcement learning demo",                # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to a specific REST endpoint
}

Create `"parameters"` field

In [5]:
# add the "parameters" block
configuration["parameters"] = {}

### C) Set Diversity Filter
During each step of Reinforcement Learning the compounds scored above `minscore` threshold are kept in memory. Those scored smiles are written out to a file in the results folder `scaffold_memory.csv`.

In [6]:
# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           #                    "NoFilter" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 25,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.4                   # the minimum similarity to be placed into the same bin
}

### D) Set Inception
* `smiles` provide here a list of smiles to be incepted 
* `memory_size` the number of smiles allowed in the inception memory
* `sample_size` the number of smiles that can be sampled at each reinforcement learning step from inception memory

In [7]:
# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": [],                          # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

### E) Set the general Reinforcement Learning parameters
* `n_steps` is the amount of Reinforcement Learning steps to perform. Best start with 1000 steps and see if thats enough.
* `agent` is the generative model that undergoes transformation during the Reinforcement Learning run.

We reccomend keeping the other parameters to their default values.

In [8]:
# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(reinvent_dir, "data/augmented.prior"), # path to the pre-trained model
    "agent": os.path.join(reinvent_dir, "data/augmented.prior"), # path to the pre-trained model
    "n_steps": 125,                        # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 128,                     # specifies how many molecules are generated per epoch
    "reset": 0,                            # if not '0', the reset the agent if threshold reached to get
                                           # more diverse solutions
    "reset_score_cutoff": 0.5,             # if resetting is enabled, this is the threshold
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

### F) Define the scoring function
We will use a `custom_product` type. The component types included are:
* `predictive_property` which is the target activity to _Aurora_ kinase represented by the predictive `regression` model. Note that we set the weight of this component to be the highest.
* `matching_substructure` the `"smiles"` field actually can take both list of SMILES or SMARTS. Normally we reccomend using one or a couple of SMARTS patterns defining your pharmacophore.
* `custom_alerts` the `"smiles"` field  also can work with SMILES or SMARTS
* `selectivity` this is the component that maximizes the `delta` between _Aurora_ and _B RAF_ kinases. Note that `delta` is undergoing transformation. Essentially if the individual prediction is higher for _Aurora_ than _B RAF_ the score will be above 0. The `weight` for this component is set lower than `predictive_property`. The relevant fields that might need to be adjusted to your needs are `activity_model_path` and `offtarget_specific_parameters`. 

Note: All models used in this example are regression models


In [9]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",              # this is our default one (alternative: "custom_sum")
    "parallel": False,                     # sets whether components are to be executed
                                           # in parallel; note, that python uses "False" / "True"
                                           # but the JSON "false" / "true"

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: an activity model
    {
        "component_type": "predictive_property", # this is a scikit-learn model, returning
                                                 # activity values
        "name": "Aurora kinase",        # arbitrary name for the component
        "weight": 6,                       # the weight ("importance") of the component (default: 1)
        "model_path": os.path.join(reinvent_dir, "data/Aurora_model.pkl"),   # absolute model path
        "smiles": [],                            # list of SMILES (not required for this component)
        "specific_parameters": {
            "transformation_type": "sigmoid",  # see description above
            "high": 9,                         # parameter for sigmoid transformation
            "low": 4,                          # parameter for sigmoid transformation
            "k": 0.25,                         # parameter for sigmoid transformation
            "scikit": "regression",            # model can be "regression" or "classification"
            "transformation": True,            # enable the transformation
            "descriptor_type": "ecfp_counts",  # sets the input descriptor for this model
            "size": 2048,                      # parameter of descriptor type
            "radius": 3,                       # parameter of descriptor type
            "use_counts": True,                # parameter of descriptor type
            "use_features": True               # parameter of descriptor type
        }
    },

    # add component: enforce the match to a given substructure
    {
        "component_type": "matching_substructure", 
        "name": "Matching substructure",       # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": ["c1ccccc1CC"],              # a match with this substructure is required
        "specific_parameters": None            # not required; note, this is "null" in JSON
    },

    # add component: enforce to NOT match a given substructure
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",               # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": [                            # specify the substructures (as list) to penalize
            "[*;r8]",
            "[*;r9]",
            "[*;r10]",
            "[*;r11]",
            "[*;r12]",
            "[*;r13]",
            "[*;r14]",
            "[*;r15]",
            "[*;r16]",
            "[*;r17]",
            "[#8][#8]",
            "[#6;+]",
            "[#16][#16]",
            "[#7;!n][S;!$(S(=O)=O)]",
            "[#7;!n][#7;!n]",
            "C#C",
            "C(=[O,S])[O,S]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
        ],
        "specific_parameters": None            # not required; note, this is "null" in JSON
    },

    # add component: calculate the QED drug-likeness score (using RDkit)
    {
      "component_type": "selectivity",
      "name": "B-RAF selectivity",
      "weight": 3,
      "smiles": [],
      "model_path": "",
      "specific_parameters": {
        "activity_model_path": os.path.join(reinvent_dir, "data/Aurora_model.pkl"),
        "offtarget_model_path": os.path.join(reinvent_dir, "data/B-RAF_model.pkl"),
        "activity_specific_parameters": {
          "high": 9,
          "low": 4,
          "k": 0.25,
          "transformation": False,
          "transformation_type": "sigmoid",
          "scikit": "regression",
          "descriptor_type": "ecfp_counts"
        },
        "offtarget_specific_parameters": {
          "high": 9,
          "low": 4,
          "k": 0.25,
          "transformation": False,
          "transformation_type": "sigmoid",
          "scikit": "regression",
          "descriptor_type": "ecfp_counts"
        },
        "delta_transformation_parameters": {
          "high": 3,
          "k": 0.25,
          "low": 0,
          "transformation": True,
          "transformation_type": "sigmoid"
        }
      }
    }]
}
configuration["parameters"]["scoring_function"] = scoring_function

#### NOTE:  Getting the selectivity score component to reach satisfactory levels is non-trivial and might take considerably higher number of steps

## 3. Write out the configuration

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [10]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## 4. Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. As we have only specified a low number of epochs (125) and all components should be fairly quick, this should not take too long in our case though.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent_shared.v2.1

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [11]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!python {reinvent_dir}/input.py {configuration_JSON_path}

In [12]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 75, 124]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

We have calculated a total of 125 epochs, let us quickly investigate how the agent fared. Below you see the print-out of the first, one from the middle and the last epoch, respectively. Note, that the fraction of valid `SMILES` is high right from the start (because we use a pre-trained prior). You can see the partial scores for each component for the first couple of compounds, but the most important information is the average score. You can clearly see how it increases over time.

In [13]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 99.2   Score: 0.0671   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-31.82    -31.82    -17.71      0.11      c1(-c2noc(C)c2C)[nH]c(N2CCc3ccccc3C2)nc(=N)n1
-42.76    -42.76    -42.76      0.00      C12(C)C(CCC3C4(C)C(C(C=O)=CC4(C)CCC3C(C)CCC(OC)C(=O)O)C1)C(C)(C)C(O)CC2
-28.86    -28.86    -11.63      0.13      FC(F)(F)c1n(CCc2ccc(C(F)(F)F)cc2)c(=O)[nH]n1
-29.12    -29.12    -22.55      0.05      c1cccc(-n2nc(-c3ccncc3)cc2C(=O)NCc2ccccc2)n1
-26.16    -26.16    -26.16      0.00      O(C1C(O)C(O)C(OC2OC(C)C(O)C(O)C2O)C(=O)O1)C(C=Cc1ccc(O)cc1)=O
-22.87    -22.87    -13.75      0.07      c12c(Cl)cccc1c1c(c(=O)[nH]2)CCSC1
-22.81    -22.81     -1.80      0.16      c1c(NCc2coc(-c3ccc(C(C)(C)C)cc3)n2)cccc1OC
-21.35    -21.35     -3.12      0.14      C1NCC(c2c(OC)ccc(Cl)c2)C1
-43.26    -43.26    -43.26      0.00      O1C(CN(C(COC)=O)C(C(C)(C)C)=O)CN(c2ccc(-c3c[nH]c(=N)cn3)c(F)c2)C1=O
-31.21    -31.21    -20.56      

## 5. Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent_shared.v2.1

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components. What you see is, that all of those depicted went up (and `Fraction_valid_SMILES` was high troughout). Not shown is the predictive model, which did not perform all that well, so you might want to consider a higher weight next time.

![](img/individual_components.png)

Also the total score increased over time.

![](img/total_score.png)

It might also be informative to look at the results from the prior (dark blue), the agent (blue) and the augmented likelihood (purple) over time.

![](img/likelihood.png)

And last but not least, there is a "Images" tab available that lets you browse through the compounds generated in an easy way. In the molecules, the substructure matches that were defined to be required are highlighted in red (if present). Also, the total scores are given per molecule.

![](img/molecules.png)

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format) and the scaffold memory (in `CSV` format).

In [14]:
!head -n 15 {output_dir}/results/memory.csv

,smiles,score,likelihood
39,c1c2c(=O)c(C(=O)NC3(C(C)C)C(=O)Nc4ccccc43)cn(CCCCC)c2c(CCC)cc1,0.37547314,-37.92856
47,c1ccc(CN(CCOc2ccc(C(c3ccc(OCCN4CCCCCC4)cc3)CCCC(C)C)cc2)C)cc1,0.3324456,-35.245316
22,c1c(C2(c3ccc(OC)c(O)c3)CCN(CCCCNC(c3ccc(-c4ccc(C(C)=O)cc4)c(OC)c3)=O)CC2)cc(NS(C)(=O)=O)c(O)c1,0.32802737,-34.53232
106,Oc1ccc(CCN2CCN(c3nsc4ccccc34)CC2)cc1NS(=O)(=O)CCC,0.32603598,-33.39078
86,CC(C(C)C)(O)Cc1c2c(ccc1O)C1C(CC2=C)C2CCC(C(C)=O)C2(C)C1,0.32173103,-42.754417
19,c1ccc(COc2ccn(CC(c3c(C)cc(CN4CCCCC4(C)C)cc3)=O)c(=O)c2)cc1,0.31507185,-26.975687
13,c1(C)c(N(C(c2c(O)cc(O)cc2)C(NC(C)(C)C)=O)C(c2cnccc2)=O)cccc1,0.3098658,-32.72461
28,C(CC1CCC2(OCC(NS(C)(=O)=O)CC2)CC1)N1CCc2ccc(C#N)cc2CC1,0.3061853,-37.942104
83,CC1(C)N(CC(C(Cc2cc(F)cc(F)c2)NC(=O)C)O)c2c(cnc(Cc3ccc(C(C)(C)C)cc3)c2)C1,0.30555668,-37.71801
106,FC1C(F)CN(C(c2cc(C(C)C)c3c(c4c(cccc4)[nH]3)c2)=O)C(CO)C1,0.30416337,-45.497562
65,N(CC(c1ccc(OC)cc1)O)C(CC(C)C)C(=O)NC(Cc1ccccc1)C(=O)NCC#N,0.3039012,-35.077744
113,c12ccccc1[nH]c